In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

In [2]:
import numpy as np
import time
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import matplotlib.pyplot as plt
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)

In [67]:
def model_RF_test(model_RF, dataX, dataY):
    print("Model performance")
    predict_data = model_RF.predict_proba(dataX)

    # Some stats
    print("Feature Importance : ")
    print(model_RF.feature_importances_) #older version of sklearn
    print("Total number of WNV occurrence in test set : " + str(len(dataY[dataY > 0])))

    print("Number of WNV occurrence the model is able to capture in test set:" + str(
        dataY[np.where(predict_data[:, 1] > 0)].sum()))
    print("check over/under prediciton" + str(predict_data[predict_data[:,1] > 0][:,1].mean() * 110000))
    print("Log loss : " + str(log_loss(dataY, predict_data)))

    print("AUC: " + str(roc_auc_score(dataY, predict_data[:,1])))

    print(
        "This is to test the performance of random forest model, ideally, the logloss is low and also it is able to "
        "capture most of the WNV occurrence")

    return None  # Check how many wnv it predicts

In [4]:
data = pd.read_sas('/home/guangya/Downloads/wnv_2245new.sas7bdat')

In [23]:
x = data.drop(columns=['wnvbinary', 'yrweeks', 'yrwksfid', 'yr_hexid', 'year', 'income1'])

In [24]:
x_selected_all = x.drop(columns=x.columns[[4, 5, 25, 26, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -6]]).values
y_selected = data['wnvbinary'].values
trainX_sel_all, testX_sel_all, trainY_sel_all, testY_sel_all = train_test_split(x_selected_all, y_selected, test_size = 0.2, random_state  = 1) # CV for model without unimportant columns

In [25]:
x_selected = x[x.drop(columns=x.columns[[4, 5, 25, 26, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -6]]).columns[[5,6,7,12,13,14,15,16,17]]].values
trainX_sel, testX_sel, trainY_sel, testY_sel = train_test_split(x_selected, y_selected, test_size = 0.2, random_state  = 1) # CV for smaller models

In [16]:
model_RF = pickle.load(open('/home/guangya/Downloads/RF_model_CV_final_11-27', 'rb'))

In [26]:
pred = model_RF.predict_proba(testX_sel_all)

In [27]:
pred

array([[1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [9.99168351e-01, 8.31649279e-04],
       ...,
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00]])

In [60]:
model_RF_test(model_RF,testX_sel_all,testY_sel_all)

Model performance
Feature Importance : 
[0.0167836  0.0166939  0.03028201 0.01733713 0.0388354  0.05063648
 0.05457309 0.05192512 0.01657915 0.01618297 0.0186369  0.0188778
 0.06026285 0.08898524 0.10783249 0.07745375 0.05806913 0.07905086
 0.01356565 0.01652435 0.01405301 0.01506404 0.01455554 0.01659846
 0.01862757 0.00018945 0.         0.02184159 0.01364752 0.01560628
 0.02072868]
Total number of WNV occurrence in test set : 164
Number of WNV occurrence the model is able to capture in test set:156.0
check over/under prediciton853.7043064365264
Log loss : 0.00630569520345608
AUC: 0.8978717545376177
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurrence


In [45]:
model_RF

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=60,
                                              max_features=5,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=5,
                                              min_samples_split=8,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=2000, n_jobs=-1,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                        

In [46]:
model_RF.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 8,
 'min_samples_split': 4,
 'n_estimators': 1200}

In [49]:
model_RF_best_2 = RandomForestClassifier(n_estimators=1200,
                                         n_jobs=-1,
                                         max_features='sqrt',
                                         max_depth= 80,
                                         min_samples_leaf = 8,
                                         min_samples_split = 4,
                                         bootstrap=True)

In [50]:
model_RF_best_2.fit(trainX_sel,trainY_sel)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=80, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [51]:
model_RF_best_1 = RandomForestClassifier(n_estimators=1200,
                                         n_jobs=-1,
                                         max_features='sqrt',
                                         max_depth=80,
                                         min_samples_leaf = 8,
                                         min_samples_split = 4,
                                         bootstrap=True)

In [52]:
model_RF_best_1.fit(trainX_sel_all,trainY_sel_all)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=80, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [56]:
model_RF_test(model_RF_best_1,testX_sel_all,testY_sel_all)

Model performance
Feature Importance : 
[0.02791739 0.02860175 0.04039711 0.03386748 0.04269938 0.0421995
 0.04133401 0.04290062 0.03439505 0.03474957 0.03549122 0.03414498
 0.04013091 0.04265336 0.04382225 0.03875162 0.03978729 0.03800087
 0.02800045 0.02745534 0.02766833 0.02505536 0.02840078 0.03126609
 0.02980734 0.00050541 0.         0.03130425 0.0287421  0.03126269
 0.02868751]
Total number of WNV occurrence in test set : 164
Number of WNV occurrence the model is able to capture in test set:156.0
check over/under prediciton154.5192569133403
Log loss : 0.004691750714525809
AUC: 0.8942706771025455
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurrence


In [64]:
x_selected = x[x.drop(columns=x.columns[[4, 5, 25, 26, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -6]]).columns[[2,4,5,6,7,12,13,14,15,16,17]]].values
trainX_sel_2, testX_sel_2, trainY_sel_2, testY_sel_2 = train_test_split(x_selected, y_selected, test_size = 0.2, random_state  = 1) # CV for smaller models

In [57]:
model_RF_test(model_RF_best_2,testX_sel,testY_sel)

Model performance
Feature Importance : 
[0.11822018 0.11606085 0.11801847 0.10684182 0.11289406 0.10880983
 0.10242753 0.10128176 0.11544551]
Total number of WNV occurrence in test set : 164
Number of WNV occurrence the model is able to capture in test set:159.0
check over/under prediciton141.1317160639655
Log loss : 0.004416525497663086
AUC: 0.906036259392426
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurrence


In [65]:
model_RF_best_3 = RandomForestClassifier(n_estimators=1200,
                                         n_jobs=-1,
                                         max_features='sqrt',
                                         max_depth=80,
                                         min_samples_leaf = 8,
                                         min_samples_split = 4,
                                         bootstrap=True)

In [66]:
model_RF_best_3.fit(trainX_sel_2,trainY_sel_2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=80, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [68]:
model_RF_test(model_RF_best_3,testX_sel_2,testY_sel_2)

Model performance
Feature Importance : 
[0.09162027 0.09602338 0.09459108 0.09212363 0.09547033 0.0850854
 0.09254329 0.08942846 0.08357565 0.0839603  0.0955782 ]
Total number of WNV occurrence in test set : 164
Number of WNV occurrence the model is able to capture in test set:161.0
check over/under prediciton172.76941043935642
Log loss : 0.004250340494764816
AUC: 0.9113654499075113
This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurrence
